In [1]:
## From https://cookbook.openai.com/examples/orchestrating_agents

In [2]:


from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel
from typing import Optional
import json

In [3]:
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
base_url = 'http://localhost:11434/v1'
api_key = 'ollama'
client  = OpenAI(base_url=base_url,api_key=api_key )
#set_tracing_disabled(disabled=True)

In [4]:
import mlflow
from pathlib import Path
current_dir = Path.cwd()
project_root = current_dir.parent
mlflow_logs = project_root / "mlflow_logs"
mlflow_logs.mkdir(exist_ok=True)
db_path = mlflow_logs / "mlflow.db"

mlflow.set_tracking_uri(f"sqlite:///{db_path.resolve()}")
mlflow.set_experiment("openai demo from website")
mlflow.openai.autolog()

2025/12/05 09:57:10 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/05 09:57:11 INFO mlflow.store.db.utils: Updating database tables
2025-12-05 09:57:11 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-05 09:57:11 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-05 09:57:11 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-05 09:57:11 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [5]:
# Customer Service Routine

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satisfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)

#@function_tool
def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    return "item_132612938"

#@function_tool
def execute_refund(item_id, reason="not provided"):

    print("Summary:", item_id, reason) # lazy summary
    return "success"


In [5]:
# Customer Service Routine

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satisfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)

#@function_tool
def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    return "item_132612938"

#@function_tool
def execute_refund(item_id, reason="not provided"):

    print("Summary:", item_id, reason) # lazy summary
    return "success"


In [6]:
import inspect

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

In [7]:
MODEL = 'qwen3:14b'

In [8]:
messages = []

#tools = [execute_refund, look_up_item]
#tool_schemas = [function_to_schema(tool) for tool in tools]
#tools = [execute_refund.params_json_schema, look_up_item.params_json_schema]
tools = [execute_refund, look_up_item]
tool_schemas = [function_to_schema(tool) for tool in tools]
response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": "Look up the black boot."}],
            tools=tool_schemas,
        )
message = response.choices[0].message



In [9]:
message

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_q7ngvm6q', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function', index=0)], reasoning='Okay, the user wants to look up the black boot. Let me check the available tools. There\'s the look_up_item function which requires a search_query. The user mentioned "black boot," so I should use that as the search query. I need to make sure the parameters are correctly formatted. The function doesn\'t require any other parameters, just the search query. So I\'ll generate the tool call with "black boot" as the search_query.\n')

In [10]:
message.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_q7ngvm6q', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function', index=0)]

In [11]:
message.tool_calls[0].function

Function(arguments='{"search_query":"black boot"}', name='look_up_item')

In [12]:
tools_map = {tool.__name__: tool for tool in tools}

def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            # add result back to conversation 
            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

Assistant: look_up_item({'search_query': 'black boot'})
